In [ ]:
import numpy as np
import string, time 
import pandas as pd 
from keras.models import Model
from keras.layers import Dense, Input, Embedding
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPool1D, Dropout, concatenate
from keras.preprocessing import text as keras_text, sequence as keras_seq
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

In [ ]:
#decorator function for calculating the total time reqired to execute various function
def calc_time(func):
  def inner(*args, **kwargs):
    st = time.time()
    result = func(*args,**kwargs)
    end = time.time()-st
    print("Total time required: {:.3f} ms".format(end * 1000))
    return result
  return inner

In [ ]:
#function is used to tokenize, create character embedding vocabulary and save the best model during the training of the model
def train_model(train,test):

  #hyper parameters
  max_features = 64
  maxlen = 512

  list_sentences_train = train["Content"].fillna("unknown").values
  list_classes = ["Label"]
  y = train[list_classes].values
  list_sentences_test = test["Content"].fillna("unknown").values

  #sequence generation
  tokenizer = keras_text.Tokenizer(num_words=None, char_level=True, oov_token='UNK')
  tokenizer.fit_on_texts(list(string.printable))

  print("length of word_index",len(tokenizer.word_index))
  print("length of word_counts",len(tokenizer.word_counts))
  print("length of index_word",len(tokenizer.index_word))
  #print(tokenizer.word_index)
  #train data
  list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
  X_t = keras_seq.pad_sequences(list_tokenized_train, maxlen=maxlen)
  
  #test data  
  list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
  X_te = keras_seq.pad_sequences(list_tokenized_test, maxlen=maxlen)

  any_category_positive = np.sum(y,1)
  print('Distribution of Total Positive Labels (important for validation)')
  print(pd.value_counts(any_category_positive))
  X_t_train, X_t_test, y_train, y_test = train_test_split(X_t, y, 
                                                          test_size = 0.2, 
                                                          stratify = any_category_positive,
                                                          random_state = 2017)

  print('Training:', X_t_train.shape)
  print('Testing:', X_t_test.shape)

  batch_size = 128 # large enough that some other labels come in
  epochs = len(X_te)

  #file path to save the best character embedding weights
  file_path="/content/drive/MyDrive/COMP-5800-YDE-Yidong-Devansh-Final-Project/EmbeddingFileVocabData/char_embedding_weights.h5"

  #checkpoint at every epoh and save the best model
  checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

  #stop training of the model if there is no further improvement
  early = EarlyStopping(monitor="val_loss", mode="min", patience=5)

  callbacks_list = [checkpoint, early]

  #character embedding model
  def build_model(conv_layers = 2, 
                dilation_rates = [0, 2, 4, 8, 16], 
                embed_size = 300):
    inp = Input(shape=(None, ))
    x = Embedding(input_dim = len(tokenizer.word_index)+1, 
                  output_dim = embed_size)(inp)
    prefilt_x = Dropout(0.25)(x)
    out_conv = []
    # dilation rate lets us use ngrams and skip grams to process 
    for dilation_rate in dilation_rates:
        x = prefilt_x
        for i in range(2):
            if dilation_rate>0:
                x = Conv1D(16*2**(i), 
                           kernel_size = 3, 
                           dilation_rate = dilation_rate,
                          activation = 'relu',
                          name = 'ngram_{}_cnn_{}'.format(dilation_rate, i)
                          )(x)
            else:
                x = Conv1D(16*2**(i), 
                           kernel_size = 1,
                          activation = 'relu',
                          name = 'word_fcl_{}'.format(i))(x)
        out_conv += [Dropout(0.5)(GlobalMaxPool1D()(x))]
    x = concatenate(out_conv, axis = -1)    
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

  #build the model
  model = build_model()

  #display model summary
  model.summary()

  model.fit(X_t_train, y_train, 
          validation_data=(X_t_test, y_test),
          batch_size=batch_size, 
          epochs=epochs, 
          shuffle = True,
          callbacks=callbacks_list)

In [ ]:
@calc_time
def main():
  
  #get the final cleaned data
  df=pd.read_csv('/content/drive/MyDrive/COMP-5800-YDE-Yidong-Devansh-Final-Project/EmbeddingFileVocabData/final_preprocessed_data_yidong_devansh_clean.csv',names=["Content","Label"])

  #split dataset into train and test
  train, test = train_test_split(df, test_size=0.3, random_state=1, shuffle=True)

  #shuffle dataset
  train = train.sample(frac=1)

  #reset index for train and test
  train.reset_index(drop=True, inplace=True)
  test.reset_index(drop=True, inplace=True)

  #this is used to find the maximum length of sentence in our data
  df["maxlen"] = df.Content.apply(lambda x: len(x.split()))
  print("Maximum length of words in a sentence",max(df["maxlen"].values))

  train_model(train,test)

In [ ]:
if __name__:main()

Maximum length of words in a sentence 3750
length of word_index 75
length of word_counts 74
length of index_word 75
Distribution of Total Positive Labels (important for validation)
0    283728
1     55891
dtype: int64
Training: (271695, 512)
Testing: (67924, 512)
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, None, 300)    22800       ['input_2[0][0]']                
                                                                                                  
 dropout_8 (Dropout)            (None, None, 300)    0           ['embedding_1[0][0]']            
                          